## 泰坦尼克号乘客数据查验

In [5]:
import pandas as pd
# 利用pandas的read_csv模块直接从互联网手机泰坦尼克号乘客数据
titanic = pd.read_csv('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.txt')

In [6]:
# 观察亲几行数据可以发现：数据种类各异，数值型、类别型，甚至还有缺失数据
titanic.head()

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male


In [7]:
titanic.shape

(1313, 11)

In [8]:
# 使用pandas，数据都转入pandas独有的dataframe格式，直接用info()查看数据的统计信息
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
row.names    1313 non-null int64
pclass       1313 non-null object
survived     1313 non-null int64
name         1313 non-null object
age          633 non-null float64
embarked     821 non-null object
home.dest    754 non-null object
room         77 non-null object
ticket       69 non-null object
boat         347 non-null object
sex          1313 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 112.9+ KB


## 使用决策树模型预测泰坦尼克号乘客的生还情况

In [9]:
# 选取 pclass, age, sex 作为决定幸免与否的特征
X = titanic[['pclass', 'age', 'sex']]
y = titanic['survived']

In [10]:
# 对当前的特征进行探查
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       633 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


### 数据预处理
** 1. age 数据列只有633个，需要补全 **  
** 2. sex 与 pclass 两个数据列的值都是类别型的，需要转化为数值特征，用0/1代替 **

In [14]:
# 缺失值用平均值替代
X['age'].fillna(X['age'].mean())

# 对补完的数据重新探查
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       1313 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


In [15]:
# 数据分割
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

D:\Program Soft\Anaconda\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [17]:
# 使用sklearn.feature_extraction中的特征转换器
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False)

In [20]:
# 将类别型的特征单独剥离出来，独成一列特征，数值型的保持不变
X_train = vec.fit_transform(X_train.to_dict(orient='record'))
print(vec.feature_names_)

['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', 'sex=female', 'sex=male']


In [21]:
# 同样需要对测试数据的特征进行转换
X_test = vec.transform(X_test.to_dict(orient='record'))
print(X_test)

[[ 18.           0.           1.           0.           0.           1.        ]
 [ 48.           1.           0.           0.           1.           0.        ]
 [ 31.19418104   1.           0.           0.           0.           1.        ]
 ..., 
 [ 20.           0.           0.           1.           0.           1.        ]
 [ 54.           1.           0.           0.           0.           1.        ]
 [ 31.19418104   0.           0.           1.           1.           0.        ]]


In [22]:
# 从sklearn.tree中导入决策树分类器
from sklearn.tree import DecisionTreeClassifier
# 使用默认配置初始化决策树分类器
dtc = DecisionTreeClassifier()
# 使用分割得到的数据进行模型学习
dtc.fit(X_train, y_train)
# 用训练好的决策树模型对测试特征数据进行预测
y_predict = dtc.predict(X_test)

## 决策树模型对泰坦尼克号乘客是否生还的预测

In [23]:
# 从sklearn.metrics导入classification_report
from sklearn.metrics import classification_report
# 输出预测准确性
print('The accuracy of Decision Tree is', dtc.score(X_test, y_test))

The accuracy of Decision Tree is 0.781155015198


In [24]:
# 输出更加详细的分类性能
print(classification_report(y_predict, y_test, target_names = ['died', 'survived']))

             precision    recall  f1-score   support

       died       0.91      0.78      0.84       236
   survived       0.58      0.80      0.67        93

avg / total       0.81      0.78      0.79       329

